In [ ]:
from urllib.request import urlretrieve
from pathlib import Path
import libarchive.public
import os
import re
from lxml.html.clean import clean_html
from lxml import etree
import json

In [ ]:
# TODO: do this for all SE sites
site_dump_url = 'https://archive.org/download/stackexchange/mathoverflow.net.7z'
site_dir = "mathoverflow"
local_filename, headers =  urlretrieve(site_dump_url)

In [ ]:
data_dir = Path("../data") / site_dir
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
poured = []
for entry in libarchive.public.file_pour(local_filename):
    poured.append(entry.pathname)
for entry_pathname in poured:
    final_pathname = data_dir / entry_pathname
    os.rename(entry_pathname, final_pathname)
    print("wrote %s" % final_pathname)


In [ ]:
# A bit of a wasteful approach, allocates 2-3 GBhis purpose, the tostring() function accepts the keyword argument with_tail:

>>> etree.tostring(br)
posts = etree.parse(str(data_dir / "Posts.xml"));

In [ ]:
re_dollar_math = re.compile('\$[^\$]+\$')

collected_data = []
rows = posts.getroot().findall('row')
print("Total posts to examine: %d" % len(rows))
for row in rows:
    post_type = row.get('PostTypeId')
    if post_type != '1' and post_type != '2':
        continue 
    post_score = row.get('Score')
    body = etree.HTML(clean_html("<div>"+row.get('Body')+"</div>")).find('body/div')
    etree.strip_elements(body, 'img', with_tail=False)
    etree.strip_elements(body, 'table', with_tail=False)

    for math in body.iterfind('.//span[@class="math-container"]'):
        # weirdly bad xpath? 
        math.text = 'mathformula'
    plain_q = ''
    for text in body.itertext():
        plain_q = plain_q + text
    plain_q = " ".join(plain_q.split())
    plain_q = re_dollar_math.sub('mathformula',plain_q)
    collected_data.append({
        'score': post_score,
        'type_id': post_type,
        'text': plain_q
    })



with open(data_dir / 'mathoverflow.json', 'w') as json_file:
  json.dump(collected_data, json_file)
